In [51]:
import numpy as np
import pandas as pd 

!pip -q install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
df_housing = pd.read_csv('https://raw.githubusercontent.com/LudwigForsberg87/Coursera_Captsone/main/London_Data.csv')#Import housing and population data csv
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population
0,City of London,-0.093510,51.51564,6506,8760
1,Barking and Dagenham,0.129506,51.54552,75829,201979
2,Barnet,-0.218210,51.61108,152945,379691
3,Bexley,0.146212,51.45821,98391,242142
4,Brent,-0.275680,51.56441,120448,324012


In [4]:
# add coloumn for Housing showing each borough's precent of total housing
df_housing['% Housing'] = round((df_housing['Housing Supply'] / df_housing['Housing Supply'].sum()) * 100,2)
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing
0,City of London,-0.093510,51.51564,6506,8760,0.18
1,Barking and Dagenham,0.129506,51.54552,75829,201979,2.11
2,Barnet,-0.218210,51.61108,152945,379691,4.26
3,Bexley,0.146212,51.45821,98391,242142,2.74
4,Brent,-0.275680,51.56441,120448,324012,3.35


In [5]:
# add coloumn for population showing each borough's precent of total population
df_housing['% Population'] = round((df_housing['Population'] / df_housing['Population'].sum()) * 100,2)
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population
0,City of London,-0.093510,51.51564,6506,8760,0.18,0.10
1,Barking and Dagenham,0.129506,51.54552,75829,201979,2.11,2.33
2,Barnet,-0.218210,51.61108,152945,379691,4.26,4.38
3,Bexley,0.146212,51.45821,98391,242142,2.74,2.79
4,Brent,-0.275680,51.56441,120448,324012,3.35,3.74


In [6]:
# calculating the difference between percent split of population and housing by borough
df_housing['% Diff'] = round((df_housing['% Population'] / df_housing['% Housing'])-1,2)
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population,% Diff
0,City of London,-0.093510,51.51564,6506,8760,0.18,0.10,-0.44
1,Barking and Dagenham,0.129506,51.54552,75829,201979,2.11,2.33,0.10
2,Barnet,-0.218210,51.61108,152945,379691,4.26,4.38,0.03
3,Bexley,0.146212,51.45821,98391,242142,2.74,2.79,0.02
4,Brent,-0.275680,51.56441,120448,324012,3.35,3.74,0.12


In [7]:
#sort descending, the boroughs with the highest percentage difference are those where they have a greater share of the population than the share of housing
df_housing=df_housing.sort_values(by=['% Diff'], ascending=False)
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population,% Diff
25,Redbridge,0.070085,51.58589,104688,296793,2.91,3.42,0.18
24,Newham,0.027369,51.53132,116979,332817,3.26,3.84,0.18
4,Brent,-0.275680,51.56441,120448,324012,3.35,3.74,0.12
16,Hillingdon,-0.441820,51.53663,110734,297735,3.08,3.43,0.11
14,Harrow,-0.336030,51.59467,91909,247130,2.56,2.85,0.11


In [8]:
# Reset the index of the dataframe and display df
df_housing.reset_index(drop=True, inplace=True)
df_housing.head()

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population,% Diff
0,Redbridge,0.070085,51.58589,104688,296793,2.91,3.42,0.18
1,Newham,0.027369,51.53132,116979,332817,3.26,3.84,0.18
2,Brent,-0.275680,51.56441,120448,324012,3.35,3.74,0.12
3,Hillingdon,-0.441820,51.53663,110734,297735,3.08,3.43,0.11
4,Harrow,-0.336030,51.59467,91909,247130,2.56,2.85,0.11


In [9]:
# choose top boroughs to perform analysis on 
df_topla=df_housing.head(7)
df_topla

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population,% Diff
0,Redbridge,0.070085,51.58589,104688,296793,2.91,3.42,0.18
1,Newham,0.027369,51.53132,116979,332817,3.26,3.84,0.18
2,Brent,-0.275680,51.56441,120448,324012,3.35,3.74,0.12
3,Hillingdon,-0.441820,51.53663,110734,297735,3.08,3.43,0.11
4,Harrow,-0.336030,51.59467,91909,247130,2.56,2.85,0.11
5,Hounslow,-0.378440,51.46243,101838,268770,2.83,3.10,0.10
6,Barking and Dagenham,0.129506,51.54552,75829,201979,2.11,2.33,0.10


In [10]:
# import csv with London undergrand stations
df_Stations = pd.read_csv('https://raw.githubusercontent.com/LudwigForsberg87/Coursera_Captsone/main/London_Stations.csv')#Import housing and population data csv
df_Stations.head()

,Station,OS X,OS Y,Latitude,Longitude,Zone,Postcode
0,Abbey Road,539081,183352,51.531952,0.003723,3,E15 3NB
1,Abbey Wood,547297,179002,51.490784,0.120272,4,SE2 9RH
2,Acton Central,520613,180299,51.508758,-0.263430,2,W3 6BH
3,Acton Main Line,520296,181196,51.516887,-0.267690,3,W3 9EH
4,Acton Town,519457,179639,51.503071,-0.280303,3,W3 8HN


In [11]:
# Drop unneeded columns
df_Stations.drop(['OS X', 'OS Y','Postcode'], axis=1, inplace=True)
df_Stations.head()

,Station,Latitude,Longitude,Zone
0,Abbey Road,51.531952,0.003723,3
1,Abbey Wood,51.490784,0.120272,4
2,Acton Central,51.508758,-0.263430,2
3,Acton Main Line,51.516887,-0.267690,3
4,Acton Town,51.503071,-0.280303,3


In [12]:
# Get the lat and long coordinates of London
address = 'London, UK'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [13]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough in zip(df_topla['Latitude'], df_topla['Longitude'], df_topla['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [14]:
CLIENT_ID = 'PMVJNGE4IECK4N13ICN5LUQB2GKTRI1YB3Y5HZYAFKSVH3DX' # your Foursquare ID
CLIENT_SECRET = 'PO3TPWPU15WTNQKEDPDJWDZU1RZH3PAZYMDI3GLVMOGOQ4JT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PMVJNGE4IECK4N13ICN5LUQB2GKTRI1YB3Y5HZYAFKSVH3DX
CLIENT_SECRET:PO3TPWPU15WTNQKEDPDJWDZU1RZH3PAZYMDI3GLVMOGOQ4JT


In [15]:
# Create Foursquare url
LIMIT=100
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=PMVJNGE4IECK4N13ICN5LUQB2GKTRI1YB3Y5HZYAFKSVH3DX&client_secret=PO3TPWPU15WTNQKEDPDJWDZU1RZH3PAZYMDI3GLVMOGOQ4JT&ll=51.5073219,-0.1276474&v=20180605&radius=500&limit=100'

In [16]:
# get London url venue results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fb4ce1c4360832ce858579a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Westminster',
  'headerFullLocation': 'Westminster, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 123,
  'suggestedBounds': {'ne': {'lat': 51.5118219045,
    'lng': -0.12043098456785678},
   'sw': {'lat': 51.5028218955, 'lng': -0.1348638154321432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ac518cdf964a520e6a520e3',
       'name': 'National Gallery',
       'location': {'address': 'Trafalgar Sq',
        'lat': 51.50887601013219,
        'lng': -0.1284778118133545,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.5088

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,National Gallery,Art Museum,51.508876,-0.128478
1,Trafalgar Square,Plaza,51.507987,-0.128048
2,East Trafalgar Square Fountain,Fountain,51.508088,-0.127700
3,Sainsbury Wing National Gallery,Art Museum,51.508384,-0.129001
4,ESPA Life at Corinthia,Spa,51.506402,-0.125114
5,Corinthia Hotel,Hotel,51.506607,-0.124460
6,Trafalgar Square Lions,Outdoor Sculpture,51.507641,-0.127888
7,National Portrait Gallery,Art Gallery,51.509438,-0.128032
8,St Martin-in-the-Fields,Church,51.508746,-0.126507
9,Nelson's Column,Monument / Landmark,51.507744,-0.127931


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [20]:
# Define function to call all above in a df format
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# Called above function to get London venue df
london_venues = getNearbyVenues(names=df_topla['Borough'],
                                   latitudes=df_topla['Latitude'],
                                   longitudes=df_topla['Longitude']
                                  )

Redbridge
Newham
Brent
Hillingdon
Harrow
Hounslow
Barking and Dagenham


In [22]:
print(london_venues.shape)
london_venues.head()

(45, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Redbridge,51.58589,0.070085,Greater London Towing,51.585744,0.071957,Transportation Service
1,Redbridge,51.58589,0.070085,Gold's Pharmacy,51.587270,0.069659,Pharmacy
2,Newham,51.53132,0.027369,The Black Lion,51.530196,0.022360,Pub
3,Newham,51.53132,0.027369,The Lord Stanley,51.531454,0.026919,Pub
4,Newham,51.53132,0.027369,BOSS Store,51.533296,0.021629,Boutique


In [23]:
# Number of venues returned for each borough
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham,2,2,2,2,2,2
Brent,12,12,12,12,12,12
Harrow,9,9,9,9,9,9
Hillingdon,4,4,4,4,4,4
Hounslow,12,12,12,12,12,12
Newham,4,4,4,4,4,4
Redbridge,2,2,2,2,2,2


In [24]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 30 uniques categories.


In [25]:
# one hot encoding to enable further analysis
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Neighborhood,Afghan Restaurant,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service,Café,Coffee Shop,...,Platform,Playground,Pub,Restaurant,Sandwich Place,Soccer Field,Sporting Goods Shop,Supermarket,Thai Restaurant,Transportation Service
0,Redbridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Redbridge,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Newham,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Newham,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Newham,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
london_onehot.shape

(45, 31)

In [27]:
# Grouped by Neighborhood to enable further analysis by frequency of venue type
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,Afghan Restaurant,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service,Café,Coffee Shop,...,Platform,Playground,Pub,Restaurant,Sandwich Place,Soccer Field,Sporting Goods Shop,Supermarket,Thai Restaurant,Transportation Service
0,Barking and Dagenham,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0
1,Brent,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.250000,...,0.000000,0.00,0.00,0.000000,0.083333,0.00,0.00,0.083333,0.000000,0.0
2,Harrow,0.111111,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.111111,...,0.111111,0.00,0.00,0.000000,0.111111,0.00,0.00,0.111111,0.111111,0.0
3,Hillingdon,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.25,0.00,0.000000,0.000000,0.25,0.25,0.000000,0.000000,0.0
4,Hounslow,0.000000,0.083333,0.083333,0.00,0.083333,0.000000,0.083333,0.000000,0.000000,...,0.000000,0.00,0.00,0.083333,0.000000,0.00,0.00,0.000000,0.000000,0.0
5,Newham,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.75,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0
6,Redbridge,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.5


In [28]:
london_grouped.shape

(7, 31)

In [29]:
# Display top 5 venues by Neighborhood
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
              venue  freq
0              Lake   0.5
1              Park   0.5
2  Asian Restaurant   0.0
3   Thai Restaurant   0.0
4       Supermarket   0.0


----Brent----
               venue  freq
0        Coffee Shop  0.25
1              Hotel  0.17
2   Pedestrian Plaza  0.08
3  Electronics Store  0.08
4               Park  0.08


----Harrow----
               venue  freq
0  Afghan Restaurant  0.11
1     Sandwich Place  0.11
2  Indian Restaurant  0.11
3           Platform  0.11
4      Grocery Store  0.11


----Hillingdon----
                        venue  freq
0         Sporting Goods Shop  0.25
1                Soccer Field  0.25
2  Construction & Landscaping  0.25
3                  Playground  0.25
4           Afghan Restaurant  0.00


----Hounslow----
               venue  freq
0  Indian Restaurant  0.17
1        Pizza Place  0.17
2              Hotel  0.08
3        Auto Garage  0.08
4       Burger Joint  0.08


----Newham----
               venue  f

In [30]:
# Function for returning n top venues in London
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# Dataframe for 10 most popular venues by Neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_venues_sorted = pd.DataFrame(columns=columns)
london_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    london_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Park,Lake,Transportation Service,Hotel,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service
1,Brent,Coffee Shop,Hotel,Electronics Store,Park,Pedestrian Plaza,Café,Indian Restaurant,Sandwich Place,Supermarket,Burger Joint
2,Harrow,Indian Restaurant,Platform,Bus Stop,Coffee Shop,Grocery Store,Thai Restaurant,Afghan Restaurant,Supermarket,Sandwich Place,Pub
3,Hillingdon,Sporting Goods Shop,Soccer Field,Playground,Construction & Landscaping,Transportation Service,Grocery Store,Asian Restaurant,Auto Garage,Boutique,Burger Joint
4,Hounslow,Indian Restaurant,Pizza Place,Hotel,Asian Restaurant,Auto Garage,Burger Joint,Restaurant,Business Service,Diner,Grocery Store


In [32]:
# k-means cluster set up to allow for clustering
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 2, 0, 2, 1, 3], dtype=int32)

In [33]:
london_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = df_topla

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_merged = london_merged.join(london_venues_sorted.set_index('Neighborhood'), on='Borough')

london_merged.head(10)

,Borough,Longitude,Latitude,Housing Supply,Population,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Redbridge,0.070085,51.58589,104688,296793,2.91,3.42,0.18,3,Transportation Service,Pharmacy,Hotel,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service,Café
1,Newham,0.027369,51.53132,116979,332817,3.26,3.84,0.18,1,Pub,Boutique,Transportation Service,Hotel,Asian Restaurant,Auto Garage,Burger Joint,Bus Stop,Business Service,Café
2,Brent,-0.275680,51.56441,120448,324012,3.35,3.74,0.12,2,Coffee Shop,Hotel,Electronics Store,Park,Pedestrian Plaza,Café,Indian Restaurant,Sandwich Place,Supermarket,Burger Joint
3,Hillingdon,-0.441820,51.53663,110734,297735,3.08,3.43,0.11,0,Sporting Goods Shop,Soccer Field,Playground,Construction & Landscaping,Transportation Service,Grocery Store,Asian Restaurant,Auto Garage,Boutique,Burger Joint
4,Harrow,-0.336030,51.59467,91909,247130,2.56,2.85,0.11,2,Indian Restaurant,Platform,Bus Stop,Coffee Shop,Grocery Store,Thai Restaurant,Afghan Restaurant,Supermarket,Sandwich Place,Pub
5,Hounslow,-0.378440,51.46243,101838,268770,2.83,3.10,0.10,2,Indian Restaurant,Pizza Place,Hotel,Asian Restaurant,Auto Garage,Burger Joint,Restaurant,Business Service,Diner,Grocery Store
6,Barking and Dagenham,0.129506,51.54552,75829,201979,2.11,2.33,0.10,4,Park,Lake,Transportation Service,Hotel,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service


In [54]:
from folium.plugins import MarkerCluster
MarkerCluster()

In [56]:
# create map for visualizing London clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Borough'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  
for lat, lng, station in zip(df_Stations['Latitude'], df_Stations['Longitude'], df_Stations['Station']):
    label = '{}'.format(station)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)

map_clusters

In [35]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Hillingdon,3.08,3.43,0.11,0,Sporting Goods Shop,Soccer Field,Playground,Construction & Landscaping,Transportation Service,Grocery Store,Asian Restaurant,Auto Garage,Boutique,Burger Joint


In [36]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Newham,3.26,3.84,0.18,1,Pub,Boutique,Transportation Service,Hotel,Asian Restaurant,Auto Garage,Burger Joint,Bus Stop,Business Service,Café


In [37]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Brent,3.35,3.74,0.12,2,Coffee Shop,Hotel,Electronics Store,Park,Pedestrian Plaza,Café,Indian Restaurant,Sandwich Place,Supermarket,Burger Joint
4,Harrow,2.56,2.85,0.11,2,Indian Restaurant,Platform,Bus Stop,Coffee Shop,Grocery Store,Thai Restaurant,Afghan Restaurant,Supermarket,Sandwich Place,Pub
5,Hounslow,2.83,3.10,0.10,2,Indian Restaurant,Pizza Place,Hotel,Asian Restaurant,Auto Garage,Burger Joint,Restaurant,Business Service,Diner,Grocery Store


In [38]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Redbridge,2.91,3.42,0.18,3,Transportation Service,Pharmacy,Hotel,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service,Café


In [39]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[0] + list(range(5, london_merged.shape[1]))]]

,Borough,% Housing,% Population,% Diff,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Barking and Dagenham,2.11,2.33,0.1,4,Park,Lake,Transportation Service,Hotel,Asian Restaurant,Auto Garage,Boutique,Burger Joint,Bus Stop,Business Service
